# data transformation

In [ ]:
import os
import pyreadstat
import pandas as pd
from datetime import datetime

# Paths to input files and output directories
current_directory = os.getcwd()
sav_file_path = os.path.join(current_directory, '01-data', 'yaleoutput.sav')
rename_excel_path = os.path.join(current_directory, '01-data', 'column_rename_mapping.xlsx')
output_excel_path = os.path.join(current_directory, '03-output', 'dfyale.xlsx')

# Read the .sav file
df, meta = pyreadstat.read_sav(sav_file_path)

# Drop the specified columns
columns_to_drop = ['starttime', 'endtime']
df = df.drop(columns=columns_to_drop)

# Load the rename mapping from the Excel file
rename_df = pd.read_excel(rename_excel_path, sheet_name="Mapping")
column_rename_mapping = dict(zip(rename_df['OldName'], rename_df['NewName']))

# Rename the columns
df.rename(columns=column_rename_mapping, inplace=True)

# Drop columns ending in '_select'
select_columns = [col for col in df.columns if col.endswith('_select')]
df.drop(columns=select_columns, inplace=True)

genderVar = df.filter(regex='^Xvar_gender')
print(genderVar.info)

# Get the current year
current_year = datetime.now().year

# Compute ages based on the current year
df['age'] = current_year - df['birthyr']

# Define age group bins with "65+" as the last group
bins = [0, 17, 24, 34, 44, 54, 64, 100]
labels = ['0-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']

# Categorize ages into the specified groups
df['Xvar_age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

# Convert all numerical columns to floats and round to one decimal place
for col in df.select_dtypes(include=['number']).columns:
    df[col] = df[col].astype(float).round(1)

# Load the Grouping sheet
grouping_df = pd.read_excel(rename_excel_path, sheet_name="Grouping")

# Mapping functions by suffix
suffix_mappings = {
    '_impact': dict(zip(grouping_df[grouping_df['Suffix'] == '_impact']['Code'], grouping_df[grouping_df['Suffix'] == '_impact']['Recode'])),
    '_confidence': dict(zip(grouping_df[grouping_df['Suffix'] == '_confidence']['Code'], grouping_df[grouping_df['Suffix'] == '_confidence']['Recode'])),
    '_important': dict(zip(grouping_df[grouping_df['Suffix'] == '_important']['Code'], grouping_df[grouping_df['Suffix'] == '_important']['Recode'])),
    '_likelihood': dict(zip(grouping_df[grouping_df['Suffix'] == '_likelihood']['Code'], grouping_df[grouping_df['Suffix'] == '_likelihood']['Recode'])),
    '_agreement': dict(zip(grouping_df[grouping_df['Suffix'] == '_agreement']['Code'], grouping_df[grouping_df['Suffix'] == '_agreement']['Recode'])),
    'Xvar_gender': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_gender']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_gender']['Recode'])),
    'Xvar_politics': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_politics']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_politics']['Recode'])),
    'Xvar_Q04_01_ed_courses_CS': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_01_ed_courses_CS']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_01_ed_courses_CS']['Recode'])),
    'Xvar_Q04_02_ed_undergrad_CS': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_02_ed_undergrad_CS']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_02_ed_undergrad_CS']['Recode'])),
    'Xvar_Q04_03_ed_Grad_CS': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_03_ed_Grad_CS']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_03_ed_Grad_CS']['Recode'])),
    'Xvar_Q04_04_ed_program_exp': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_04_ed_program_exp']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_04_ed_program_exp']['Recode'])),
    'Xvar_Q04_05_ed_none': dict(zip(grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_05_ed_none']['Code'], grouping_df[grouping_df['Suffix'] == 'Xvar_Q04_05_ed_none']['Recode']))
}

# Function to recode columns based on suffix
def apply_mapping(df, mapping_dict, suffix):
    for col in df.columns:
        if col.endswith(suffix):
            df[col] = df[col].replace(mapping_dict)

# Apply mappings for each suffix
for suffix, mapping_dict in suffix_mappings.items():
    apply_mapping(df, mapping_dict, suffix)

# List of columns to be one-hot encoded
columns_to_encode = ['Xvar_gender','Xvar_age_group','Xvar_politics', 'Xvar_Q04_01_ed_courses_CS', 'Xvar_Q04_02_ed_undergrad_CS', 'Xvar_Q04_03_ed_Grad_CS', 'Xvar_Q04_04_ed_program_exp', 'Xvar_Q04_05_ed_none']

# Apply one-hot encoding (dummy variables) without converting to integers
df_dummies = pd.get_dummies(df, columns=columns_to_encode, drop_first=False).astype(float)
# Select specific numerical columns to round
df_dummies[col] = df_dummies[col].astype(float).round(1)

columns_to_drop = ['birthyr', 'age','race','educ','marstat','employ','faminc_new','pid7','inputstate','votereg','ideo5','newsint','religpew','pew_churatd','pew_bornagain','pew_religimp','pew_prayer','Q03new_treat','q05b_treat','q12a_treat','q12_treat','q15_treat']

df_dummies = df_dummies.drop(columns=columns_to_drop, errors='ignore')

genderVar = df_dummies.filter(regex='^Xvar_gender')
print(genderVar.info)

# Save the modified DataFrame to an Excel file
df_dummies.to_excel(output_excel_path, index=False)

print(f"DataFrame with renamed columns is saved to {output_excel_path}")


# Model section

In [6]:
import pandas as pd
from sklearn.decomposition import PCA
from semopy import Model
import pydot
import os

# Load the original data
current_directory = os.getcwd()
survey = os.path.join(current_directory, '03-output', 'dfyale.xlsx')
sheet_name = 'Sheet1'
df = pd.read_excel(survey, sheet_name=sheet_name)

# Identify independent columns (features) for PCA
feature_columns = [col for col in df.columns if col.startswith('Xvar')]

# Apply PCA to reduce features
num_components = 5  # Adjust based on how much variance you want to capture
pca = PCA(n_components=num_components)
pca_result = pca.fit_transform(df[feature_columns])

# Create a DataFrame for the PCA-transformed features
pca_columns = [f'PC{i+1}' for i in range(num_components)]
pca_df = pd.DataFrame(pca_result, columns=pca_columns)

# Combine with dependent variables
dependent_columns = ['Y_Q05_SupportAI', 'Y_Q17_SupportDevHighlevelAI']
final_df = pd.concat([pca_df, df[dependent_columns]], axis=1)

# Example SEM model with reduced PCA features
model_desc = f"""
# Structural Model (hypothetical)
Y_Q05_SupportAI ~ {' + '.join(pca_columns)}
Y_Q17_SupportDevHighlevelAI ~ {' + '.join(pca_columns)}
"""

# Create the SEM model and load the dataset
model = Model(model_desc)
model.load_dataset(final_df)

# Optimize the model
model.fit()

# Create a graph using pydot
graph = pydot.Dot(graph_type='digraph')

# Add nodes (features) and edges (relationships) manually
nodes = pca_columns + ['Y_Q05_SupportAI', 'Y_Q17_SupportDevHighlevelAI']
for node in nodes:
    graph.add_node(pydot.Node(node))

# Add relationships (edges) between PCA components and dependent variables
dependent_edges = [
    ('Y_Q05_SupportAI', pca_columns),
    ('Y_Q17_SupportDevHighlevelAI', pca_columns)
]

# Add edges to the graph
for target, sources in dependent_edges:
    for source in sources:
        graph.add_edge(pydot.Edge(source, target))

# Save the graph as a PNG image
output_directory = os.path.join(current_directory, '04-summary')
os.makedirs(output_directory, exist_ok=True)
graph_output_path = os.path.join(output_directory, 'sem_model_graph.png')
graph.write_png(graph_output_path)

print(f"Model graph saved to: {graph_output_path}")


Model graph saved to: /Users/danramirez/mbs-structural-equation-modeling/04-summary/sem_model_graph.png
